In [6]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [7]:
import pandas as pd

df = pd.read_csv('../data/isear_processed.csv',index_col=0, names=['text','emotion'], header=0)

In [8]:
df.head()

,text,emotion
0,"During the period of falling in love, each tim...",joy
1,When I was involved in a traffic accident.,fear
2,When I was driving home after several days of...,anger
3,When I lost the person who meant the most to me.,sadness
4,The time I knocked a deer down - the sight of ...,disgust


In [9]:
df['emotion'].value_counts()

sadness    1096
anger      1096
disgust    1096
shame      1096
fear       1095
joy        1094
guilt      1093
Name: emotion, dtype: int64

In [10]:
emotion_dict = {}
labels = df['emotion']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
print('Creating the emotion dict to encode the label')

Creating the emotion dict to encode the label


In [5]:
texts = df['text']

In [6]:
import numpy as np

encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

seq_length = 50

tokenizer = Tokenizer(num_words=10000)

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = pad_sequences(sequences, maxlen=50)

Using TensorFlow backend.
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qi

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [9]:
import torch

train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train).type(torch.LongTensor), torch.from_numpy(np.array(y_train)).type(torch.LongTensor))
val_data = TensorDataset(torch.from_numpy(X_test).type(torch.LongTensor), torch.from_numpy(y_test).type(torch.LongTensor))

# dataloaders
batch_size = 64

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

In [16]:
from basic_transformer import Transformer

num_words = len(tokenizer.word_index) + 1
num_classes = len(labels.unique())

model = Transformer(50,2,2,num_words, seq_length,num_classes)

print(model)

Transformer(
  (token_embedding): Embedding(9064, 50)
  (position_embedding): Embedding(50, 50)
  (transfo_blocks): Sequential(
    (0): TransformerBlock(
      (self_attention): SelfAttention(
        (to_keys): Linear(in_features=50, out_features=100, bias=True)
        (to_queries): Linear(in_features=50, out_features=100, bias=True)
        (to_values): Linear(in_features=50, out_features=100, bias=True)
        (merge_heads): Linear(in_features=100, out_features=50, bias=True)
      )
      (norm1): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=50, out_features=100, bias=True)
        (1): ReLU()
        (2): Linear(in_features=100, out_features=50, bias=True)
      )
    )
    (1): TransformerBlock(
      (self_attention): SelfAttention(
        (to_keys): Linear(in_features=50, out_features=100, bias=True)
        (to_queries): Linear(in_features=50,

In [17]:
import torch.nn as nn
from torch import optim

criterion = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

In [20]:
EPOCHS = 10
for e in range(EPOCHS):
    model.train(True)

    for text_sequences, targets in train_loader:

        opt.zero_grad()

        out = model(text_sequences)
        loss = criterion(out, targets)

        loss.backward()

        # clip gradients
        # - If the total gradient vector has a length > 1, we clip it back down to 1.
        nn.utils.clip_grad_norm_(model.parameters(), 1)

        opt.step()
        #sch.step()


    print("Epoch: {}/{}  ".format(e+1, EPOCHS),
            "Loss: {:.6f}  ".format(loss.item()))

Epoch: 1/10   Loss: 1.944658  
Epoch: 2/10   Loss: 1.943278  
Epoch: 3/10   Loss: 1.946526  
Epoch: 4/10   Loss: 1.943841  
Epoch: 5/10   Loss: 1.950224  
Epoch: 6/10   Loss: 1.939775  
Epoch: 7/10   Loss: 1.937046  
Epoch: 8/10   Loss: 1.945402  
Epoch: 9/10   Loss: 1.951010  
Epoch: 10/10   Loss: 1.946500  
